In [ ]:
!pip install -U diffusers
!pip install huggingface_hub



In [ ]:
!huggingface-cli login

In [ ]:
import os
import gc
import pandas as pd
import torch
from tqdm import tqdm
from diffusers import StableDiffusionXLPipeline


# 1. Configuring the environment and defining paths

csv_path = "mscoco-blip-vit_captions.csv"   # <-- your captions file
model_name = "SDXL"
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
save_dir = model_name
os.makedirs(save_dir, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"


# 2. Loading the Stable Diffusion XL model

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16
).to(device)

# memory-saving modes
pipe.enable_attention_slicing()
pipe.enable_vae_slicing()
pipe.enable_sequential_cpu_offload()


# 3. Reading CSV in Chunks (skip first 'skip_rows' rows)

chunksize = 50   # how many rows to process at once
skip_rows = 0   # number of rows you want to skip this was done to avoid session crash because of all ram usage

print(f"Reading {csv_path} in chunks of {chunksize} rows, skipping first {skip_rows} rows...")

reader = pd.read_csv(csv_path, chunksize=chunksize, skiprows=range(1, skip_rows+1))
# NOTE: range(1, N+1) skips the first N rows but keeps the header

for chunk in reader:
    for row in tqdm(chunk.itertuples(), total=len(chunk), desc=f"Generating with {model_name}"):
        image_name = row.image_name
        caption = row.mscoco_caption   # <-- caption text


        # 4. Generating Image

        image = pipe(caption, height=512, width=512).images[0]


        # 5. Saving Image

        save_path = os.path.join(save_dir, f"{model_name}-{image_name}.jpeg")
        image.save(save_path, "JPEG")


        # 6. Freeing the Memory

        del image
        torch.cuda.empty_cache()
        gc.collect()

print(" All images generated and saved!")
import shutil # downloading the folder

# Zip the folder
shutil.make_archive("SDXL", 'zip', "SDXL")

# Download to your local machine
from google.colab import files
files.download("SDXL.zip")
